Imports

In [9]:
!pip install nnsight

In [10]:
!pip install sae_lens

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4


In [11]:
!pip install --upgrade numpy


  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
automated-interpretability 0.0.8 requires numpy<2.0.0,>=1.24.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.


In [30]:
# Import necessary libraries
from nnsight import LanguageModel
import torch
import pandas as pd

# Initialize the language model with specific parameters
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="cuda:0", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False)



from sae_lens import SAE
layer = 12

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb",
    sae_id = f"blocks.{layer}.hook_resid_post",
    device = 'cuda:0'
)
# Load the SAE weights using safetensors
# from safetensors import safe_open
# with safe_open("/home/ashater/work/sae.safetensors", framework="pt") as st:
#     w_dec = st.get_tensor("W_dec").to('cuda:1')





In [31]:

from huggingface_hub import hf_hub_download

# Define the repository ID and filename
repo_id = "jbloom/Gemma-2b-Residual-Stream-SAEs"
filename = "sae_weights.safetensors"

# Download the file


filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.12.hook_resid_post_16384")

# Print the downloaded file path
print(filepath)

/root/.cache/huggingface/hub/models--jbloom--Gemma-2b-Residual-Stream-SAEs/snapshots/2e64e9127f413318e0500a6ba3981483f251bcee/gemma_2b_blocks.12.hook_resid_post_16384/sae_weights.safetensors


In [32]:
# Load the SAE weights using safetensors
with safe_open(filepath, framework="pt") as st:
    w_dec = st.get_tensor("W_dec").to('cuda:0')

In [33]:
import torch
#torch.cuda.is_available()

#torch.cuda.device_count()
#len(docs)
torch.cuda.set_device(0)
torch.cuda.current_device()


0

In [34]:
#sae.W_dec == w_dec

cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos(w_dec[471,:], sae.W_dec).topk(100).indices

tensor([  471,  5567, 12267,  1271,   125,  6172,  3849, 11287, 11984, 15389,
         1738, 14166,  2724, 11198,  2401,  3469,  2800,  6135,  4142, 14455,
         8743, 16260, 13655,  8987,  1900,   475,  6439,  8133, 15013, 15650,
         7723,  6443,  4464,  6137,  7224,  7192,  6946,  5821, 15033,  1179,
          651,  5972,   115,  1729, 13180,  6421,  7071,  6386,   496, 12368,
        12540,  7448,  1378, 14433, 14164,  5279,  1191, 14866, 14503,  6020,
         2580,  3484,  1740, 11450, 10285, 10713,  4892,  5659, 11635,  9149,
         1840,  5987,  8838,  6341, 14719,  8456, 10569, 12884,  7395,  8370,
        12425, 13055,  2184,  5801,  9397,   519, 12207, 12601,  4428, 14192,
         6338, 14951,  2434,  7022,  6694,  6525,  9327,  9807,   329, 12589],
       device='cuda:0')

Parameters

In [35]:
#@title Self-explanation in 18 lines
feature = 471  #@param {type: "integer"}
#feature = 5892  #@param {type: "integer"}
# feature = 9415
# feature = 3629
scale = 55.5 #@param {type: "number"}
se_demo = True  #@param {type: "boolean"}
max_new_tokens = 200  #@param {type: "integer"}
n_generate = 1  #@param {type: "integer"}
vals = ['positive', 'neutral', 'negative']
samples = pd.read_excel('/home/ashater/work/sae/sentence_sentiment_gpt4o.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '/home/ashater/work/sae/sentence_sentiment_gpt4o.xlsx'

## Main code prompt


In [36]:
full_res = []
#v = range(0,10)
#for ismpl in v:


    # prompt = """<start_of_turn>user\nCan you rate  the following sentnece as a financial credit sentiment, very positive,
    # somewhat positive, neutral, somewhat negative,
    # very negative? """ + ismpl + '<end_of_turn>\n<start_of_turn>model\nThe sentiment of the setence is "X"'


prompt = """<start_of_turn>user\n The president anounced new tariffs but at the same time promised tax cuts.<end_of_turn>\n<start_of_turn>model\n
How would that affect the economy of the country? """


#prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

# prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
    # vector steering section
    # for feature_i in topics:
    vector = sae.W_dec[[feature]]
    vector = vector / vector.norm()
    vector = vector * scale
    # setting vector into layer 2
    # for position in positions:
    #   model.model.layers[2].output[0][:, position] = vector

    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[12].output.save()

features = sae.encode(resid[0])
summed_activations = features.abs().sum(dim=1) # Sort by max activations
summed_activations = features
# top_activations_indices = summed_activations.topk(200).indices # Get indices of top 20
top_activations_indices = summed_activations.indices # Get indices of top 20


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    print(prompt + "|"+ s + "|" + str(top_activations_indices))
    # print(str(top_activations_indices))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

<start_of_turn>user
 The president anounced new tariffs but at the same time promised tax cuts.<end_of_turn>
<start_of_turn>model
 
How would that affect the economy of the country? |'\n\nThe announcement of new tariffs could lead to:\n\n- Increase in imports, driving up prices for consumers and businesses\n- Fall in imports, leading to lower prices for consumers and businesses\n- An impact on production and investment\n- Potential for job losses and increased unemployment\n- Strain on the balance of payments, as the country will be spending more on imports than it is receiving in exports\n\n The promise of tax cuts could lead to:\n\n- Lower prices for consumers and businesses\n- Increased investment\n- More money in the economy'|<built-in method indices of Tensor object at 0x7cc325ca6570>


354	Financial management and allocation of funds for specific purposes.
2105	The concept of being part of or related to something.
4218	Graphical analysis, convergence points, equilibrium, minimum and maximum values, thresholds.
5239	Positive trends and growth in economic indicators and stock markets.
6545	Types and structures of companies, particularly publicly traded and holding companies.
7529	Financial terminology related to price changes and asset value fluctuations.
8461	The documents discuss the nuances of suggesting, recommending, or indicating.
11614	Mining operations, financial deposits, and ongoing projects or undertakings.
11916	Financial performance metrics and valuation indicators for companies and stocks.
14291	Stock market indices and their role in performance measurement.
15399	Investment, business involvement, management, trading, profit, engagement, control, interest.


## summed activations across tokens, finance features

In [37]:
#summed_activations[summed_activations > 10].shape
idc = summed_activations.topk(1000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[True, False, False, False, False, False, False, False, False, False, False]

## Get features for each token

In [38]:
idc = summed_activations.mean(dim=1).topk(5000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[False, True, False, False, False, False, False, True, False, False, False]